In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

%matplotlib inline

# For preprocessing
import csv
from os import path
from pathlib import Path

# Set the project root folder
root_dir = Path().absolute()

In [2]:
try:
    data = pd.read_csv('../data/accre-jobs-2020.csv')
except Exception as error:
    print(error)

Error tokenizing data. C error: Expected 13 fields in line 3461, saw 15



In [3]:
try:
    # The data might have already been pre-processed previoulsy
    # If not, this line would generate a “No such file” error so we would fallback to the exception below
    data = pd.read_csv('../data/accre-jobs-2020-processed.csv')
    print('Reading from previously processed file...')
    display(data.head())
except FileNotFoundError:
    print('Preprocessed file not found. Creating a new one. Please wait...')
    # Open 2 files: One is the source and one is the destination
    with open(path.join(root_dir, '..', 'data', 'accre-jobs-2020.csv'), mode='r') as source,\
    open(path.join(root_dir, '..', 'data', 'accre-jobs-2020-processed.csv'), mode='w', newline='') as destination:
        # Set file reader and file writer on the source and destination
        reader = csv.reader(source)
        writer = csv.writer(destination)
        # Go through each line in the source
        for line in reader:
            # Create a new matching line with the last column within quotes
            # This will make it into a single column if there are more commas
            newline = line[:12] + [','.join(line[12:])]
            # Write to the destination file
            writer.writerow(newline)
    # Try importing as normal again on the newly created destination file
    data = pd.read_csv('../data/accre-jobs-2020-processed.csv')
    display(data.head())
except Error as error:
    print('Something went wrong:', error)

Reading from previously processed file...


,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


In [4]:
accre_jobs_2020 = pd.read_csv('../data/accre-jobs-2020-processed.csv').rename(columns = str.lower)
accre_jobs_2020 = accre_jobs_2020[accre_jobs_2020.state.str.contains('RUNNING') == False]

In [5]:
accre_jobs_2020.head()

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440


accre_jobs_2020 = pd.read_csv('../data/accre-jobs-2020.csv', error_bad_lines = False)

accre_jobs_2020 = pd.read_csv('../data/accre-jobs-2020.csv', nrows = 3459)

line 612270: expected 13 fields, saw 100\nSkipping line 612271

accre_jobs_2020 = pd.read_csv('../data/accre-jobs-2020.csv', usecols = [0,1,2,3,4,5,6,7,8,9,10,11,12]).rename(columns = str.lower)

In [6]:
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312


In [7]:
accre_jobs_2020['state'].value_counts()

COMPLETED              3804644
CANCELLED                 6107
FAILED                    1406
CANCELLED by 686562        681
OUT_OF_MEMORY               94
TIMEOUT                     80
CANCELLED by 505355         63
CANCELLED by 855431         17
CANCELLED by 782611         13
CANCELLED by 9206            9
CANCELLED by 397600          7
CANCELLED by 200557          6
CANCELLED by 0               5
CANCELLED by 486541          4
CANCELLED by 199766          4
CANCELLED by 454080          4
CANCELLED by 9201            3
CANCELLED by 649319          3
CANCELLED by 483348          3
CANCELLED by 9202            3
CANCELLED by 546080          2
CANCELLED by 199066          2
CANCELLED by 666860          2
CANCELLED by 515423          2
CANCELLED by 124006          2
CANCELLED by 895426          2
CANCELLED by 791651          2
CANCELLED by 693461          2
CANCELLED by 90423           2
CANCELLED by 649321          1
CANCELLED by 337422          1
CANCELLED by 503585          1
CANCELLE

In [8]:
accre_cancelled_by = accre_jobs_2020[accre_jobs_2020['state'] == 'CANCELLED by 686562']
accre_cancelled_by

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
292610,17494426,sharlin,page,10240Mn,0,7-00:00:00,00:00:14,1,1,production,0:0,CANCELLED by 686562,cn1089
292615,17494431,sharlin,page,10240Mn,0,7-00:00:00,00:00:14,1,1,production,0:0,CANCELLED by 686562,cn1128
292616,17494432,sharlin,page,10240Mn,0,7-00:00:00,00:00:06,1,1,production,0:0,CANCELLED by 686562,cn1127
292618,17494434,sharlin,page,10240Mn,0,7-00:00:00,00:00:07,1,1,production,0:0,CANCELLED by 686562,cn1122
292623,17494439,sharlin,page,10240Mn,0,7-00:00:00,00:00:06,1,1,production,0:0,CANCELLED by 686562,cn425
...,...,...,...,...,...,...,...,...,...,...,...,...,...
309575,17522665,sharlin,page,10240Mn,0,7-00:00:00,00:00:01,1,1,production,0:0,CANCELLED by 686562,cn442
309576,17522666,sharlin,page,10240Mn,0,7-00:00:00,00:00:00,1,1,production,0:0,CANCELLED by 686562,cn1316
309578,17522668,sharlin,page,10240Mn,0,7-00:00:00,00:00:01,1,1,production,0:0,CANCELLED by 686562,cn1127
309579,17522669,sharlin,page,10240Mn,0,7-00:00:00,00:00:05,1,1,production,0:0,CANCELLED by 686562,cn1127


In [9]:
accre_jobs_2020['account'].nunique()

175

In [10]:
accre_jobs_2020['account'].value_counts()

cms          806456
cep          609282
summer       380840
galia        198036
carrot       192551
              ...  
with              1
tree              1
purple            1
president         1
pearl             1
Name: account, Length: 175, dtype: int64

In [11]:
accre_jobs_2020['nodes'].value_counts()

1      3807797
2         2540
3          885
4          604
5          417
        ...   
39           1
38           1
37           1
109          1
500          1
Name: nodes, Length: 83, dtype: int64

In [12]:
accre_jobs_2020[accre_jobs_2020.account.str.contains('cms')]

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
2125,17032244,cms,cmspilot,28000Mn,8320.07M,2-00:00:00,1-23:31:46,1,4,production,0:0,COMPLETED,cn1205
2126,17032568,cms,cmspilot,28000Mn,11954.84M,2-00:00:00,1-23:33:13,1,4,production,0:0,COMPLETED,cn1492
2127,17032569,cms,cmspilot,28000Mn,10498.84M,2-00:00:00,1-23:36:36,1,4,production,0:0,COMPLETED,cn1497
2128,17032916,cms,cmspilot,21875Mn,11728.02M,2-00:00:00,1-23:27:55,1,8,nogpfs,0:0,COMPLETED,ng1037
2129,17032920,cms,cmspilot,94451Mn,20659.08M,2-00:00:00,1-23:16:04,1,8,nogpfs,0:0,COMPLETED,ng691
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816215,24173661,cmsadmin,autocms,4096Mc,860.03M,12:00:00,00:03:50,1,1,production,0:0,COMPLETED,cn394
3816221,24173686,cmsadmin,lcgadmin,2000Mn,0.09M,2-00:00:00,00:02:29,1,1,sam,0:0,COMPLETED,vm-cms-sam-pri
3816233,24173689,cmsadmin,lcgadmin,2000Mn,0.09M,2-00:00:00,00:02:11,1,1,sam,0:0,COMPLETED,vm-cms-sam-sec
3816240,24173711,cmsadmin,lcgadmin,2000Mn,0.09M,2-00:00:00,00:01:29,1,1,sam,0:0,COMPLETED,vm-cms-sam-pri


In [13]:
cms_jobs = accre_jobs_2020[accre_jobs_2020.account.str.contains('cms')]

In [14]:
cms_jobs['account'].value_counts()

cms            806456
cms_lowprio    135682
cmsadmin       102589
cms_gpu_acc      1196
Name: account, dtype: int64

In [15]:
cms_jobs.shape

(1045923, 13)

In [16]:
cms_jobs['nodes'].value_counts()

1    1045922
5          1
Name: nodes, dtype: int64

In [17]:
cms_Mc = cms_jobs[cms_jobs.reqmem.str.contains('Mc')]
cms_Mc

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
7865,17092491,cmsadmin,autocms,2048Mc,819.12M,12:00:00,04:00:11,1,1,production,0:0,COMPLETED,cn1134
8186,17094019,cmsadmin,autocms,2048Mc,789.33M,12:00:00,00:08:01,1,1,production,0:0,COMPLETED,cn1088
8225,17094164,cmsadmin,autocms,2048Mc,805.13M,12:00:00,00:06:19,1,1,production,0:0,COMPLETED,cn913
8251,17094286,cmsadmin,autocms,2048Mc,863.34M,12:00:00,00:03:24,1,1,production,0:0,COMPLETED,cn1081
8327,17094473,cmsadmin,autocms,2048Mc,870.35M,12:00:00,00:04:48,1,1,production,0:0,COMPLETED,cn1278
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816072,24173338,cmsadmin,autocms,4096Mc,872.41M,12:00:00,00:04:17,1,1,production,0:0,COMPLETED,cn394
3816132,24173475,cms,lailah,1024Mc,62.46M,1-20:00:00,00:00:47,1,1,production,0:0,COMPLETED,cn394
3816144,24173491,cmsadmin,autocms,4096Mc,869.54M,12:00:00,00:04:19,1,1,production,0:0,COMPLETED,cn421
3816175,24173567,cms,lailah,1024Mc,1014.77M,1-20:00:00,00:05:20,1,1,production,0:0,COMPLETED,cn394


In [18]:
accre_jobs_2020[accre_jobs_2020["account"]].str.match('\d+').value_counts()

KeyError: "None of [Index(['treviso', 'treviso', 'treviso', 'treviso', 'treviso', 'treviso',\n       'treviso', 'treviso', 'treviso', 'treviso',\n       ...\n       'portabella', 'portabella', 'portabella', 'portabella', 'portabella',\n       'portabella', 'portabella', 'portabella', 'portabella', 'portabella'],\n      dtype='object', length=3813182)] are in the [columns]"

In [19]:
accre_jobs_2020['usedmem'].str[-1].value_counts()

M    3213712
0     599470
Name: usedmem, dtype: int64

In [20]:
accre_mem_zeros = accre_jobs_2020[(accre_jobs_2020['usedmem'] == '0') & (accre_jobs_2020['state'] == 'COMPLETED')]
accre_mem_zeros

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist
4805,17072928,endive,bennett,8192Mn,0,5-00:00:00,00:00:25,1,1,turing,0:0,COMPLETED,gpu0039
4806,17072929,endive,bennett,8192Mn,0,5-00:00:00,00:00:34,1,1,turing,0:0,COMPLETED,gpu0039
4807,17072930,endive,bennett,8192Mn,0,5-00:00:00,00:00:23,1,1,turing,0:0,COMPLETED,gpu0039
4809,17072932,endive,bennett,8192Mn,0,5-00:00:00,00:01:05,1,1,turing,0:0,COMPLETED,gpu0039
4810,17072933,endive,bennett,8192Mn,0,5-00:00:00,00:00:27,1,1,turing,0:0,COMPLETED,gpu0039
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816245,24173761_4,portabella,vennie,16384Mn,0,1-00:00:00,00:00:36,1,1,production,0:0,COMPLETED,cn1135
3816258,24173800_3,portabella,vennie,16384Mn,0,1-00:00:00,00:00:37,1,1,production,0:0,COMPLETED,cn1133
3816259,24173800_4,portabella,vennie,16384Mn,0,1-00:00:00,00:00:37,1,1,production,0:0,COMPLETED,cn1092
3816260,24173800_5,portabella,vennie,16384Mn,0,1-00:00:00,00:00:37,1,1,production,0:0,COMPLETED,cn1135


# what??

In [21]:
accre_mem_zeros['account'].value_counts()

cep              143534
orange            81295
cms               74671
boysenberries     29640
bulk              29224
                  ...  
president             1
discovery             1
shell                 1
flamingo              1
romanesco             1
Name: account, Length: 148, dtype: int64

In [22]:
accre_jobs_2020['account'].value_counts()

cms          806456
cep          609282
summer       380840
galia        198036
carrot       192551
              ...  
with              1
tree              1
purple            1
president         1
pearl             1
Name: account, Length: 175, dtype: int64

In [23]:
accre_mem_zeros['partition'].value_counts()

production        537051
nogpfs             44153
pascal              5982
turing              4238
maxwell             1062
debug                686
sam                   99
cgw-dougherty1         9
Name: partition, dtype: int64

accre_mem_zeros['partition'].plot(kind = 'bar')

In [24]:
accre_jobs_2020[["exitcode_user", "exitcode_error"]] = accre_jobs_2020["exitcode"].str.split(":", expand=True)


In [25]:
accre_jobs_2020.head(3) 

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0


In [27]:
accre_jobs_2020['used_mb_per_node'] = accre_jobs_2020['usedmem'].str.strip('M').astype(float)
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error,used_mb_per_node
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0,65973.49
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0,67181.12
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0,69111.86
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,0,0,65317.33
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,0,0,65876.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432,0,0,23269.08
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440,0,0,20879.81
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,30042.68
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,31067.75


In [28]:
accre_jobs_2020['used_mb_per_core'] = (accre_jobs_2020['used_mb_per_node'])/(accre_jobs_2020['cpus']*accre_jobs_2020['nodes'])
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error,used_mb_per_node,used_mb_per_core
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0,65973.49,2748.895417
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0,67181.12,2799.213333
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0,69111.86,2879.660833
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,0,0,65317.33,2721.555417
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,0,0,65876.11,2744.837917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432,0,0,23269.08,11634.540000
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440,0,0,20879.81,10439.905000
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,30042.68,15021.340000
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,31067.75,15533.875000


In [31]:
accre_jobs_2020['reqmem_mc'] = accre_jobs_2020[accre_jobs_2020['reqmem'].str.contains('Mc')]
accre_jobs_2020['reqmem_mn'] = accre_jobs_2020[accre_jobs_2020['reqmem'].str.contains('Mn')]
accre_jobs_2020

,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,exitcode_user,exitcode_error,used_mb_per_node,used_mb_per_core,reqmem_mc,reqmem_mn
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,0,0,65973.49,2748.895417,NaN,15925210
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,0,0,67181.12,2799.213333,NaN,15861126
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,0,0,69111.86,2879.660833,NaN,15861125
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,0,0,65317.33,2721.555417,NaN,16251645
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,0,0,65876.11,2744.837917,NaN,16251646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816283,24173815_6,portabella,vennie,32768Mn,23269.08M,1-00:00:00,00:01:36,1,2,production,0:0,COMPLETED,cn432,0,0,23269.08,11634.540000,NaN,24173815_6
3816284,24173815_7,portabella,vennie,32768Mn,20879.81M,1-00:00:00,00:01:37,1,2,production,0:0,COMPLETED,cn440,0,0,20879.81,10439.905000,NaN,24173815_7
3816285,24173815_8,portabella,vennie,32768Mn,30042.68M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,30042.68,15021.340000,NaN,24173815_8
3816286,24173815_9,portabella,vennie,32768Mn,31067.75M,1-00:00:00,00:02:27,1,2,production,0:0,COMPLETED,cn312,0,0,31067.75,15533.875000,NaN,24173815_9


In [ ]:
accre_jobs_2020['used_mb_per_core'] = (accre_jobs_2020['used_mb_per_node'])/(accre_jobs_2020['cpus']*accre_jobs_2020['nodes'])
accre_jobs_2020